In [1]:
import pandas as pd
import numpy as np
import os, sys
import re,glob
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from tqdm import tqdm, trange

In [2]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rc("font",family="Arial")

In [3]:
df = pd.read_csv("terminal_info_subtype22.csv",index_col=0)

In [4]:
color_dict = {'Adcyap1': 'red', 'Tac2': 'cyan', 'Nts': 'navy','Pdyn': 'darkgray'}

In [5]:
# subtype22
for region in [["AMd","AMv","AD","AV","IAM","IAD","LD"]]:
        dfplot = df.loc[(df.region.isin(region))]
        dfplot2 = dfplot.pivot_table(index="filename",values=["soma_AP","soma_DV","soma_ML","ter_AP","ter_DV","ter_ML"],aggfunc="mean")
        dfplot2["color"] = [color_dict[item.split("_")[2]] for item in dfplot2.index.to_list()]
        fig, axs = plt.subplots(3, 3, figsize=(15, 12))
        for i,soma_label in enumerate(["soma_AP","soma_DV","soma_ML"]):
                for j,ter_label in enumerate(["ter_AP","ter_DV","ter_ML"]):
                        x = dfplot2[soma_label]
                        y = dfplot2[ter_label]

                        regression_result = linregress(x, y)
                        axs[i, j].scatter(x=x, y=y, s=10,c= dfplot2["color"])
                        axs[i, j].plot([min(x), max(x)],
                                [min(x) * regression_result.slope + regression_result.intercept,
                                max(x) * regression_result.slope + regression_result.intercept],
                                c='black',
                                linewidth=1.5,
                                dashes=[4, 2]
                                )
                        textstr = '\n'.join(['r={:.3f}'.format(regression_result.rvalue),
                                'p={:.3e}'.format(regression_result.pvalue)])
                        axs[i, j].text(x = 0.02,
                                y = 0.9,   
                                s = textstr,
                                transform=axs[i, j].transAxes, 
                                fontsize=12,
                                verticalalignment='top'
                                )

                        if soma_label.__contains__("ML"):
                                axs[i, j].invert_xaxis()

                        if ter_label.__contains__("ML"):
                                axs[i, j].invert_yaxis()


                        axs[i, j].set_title(soma_label+"-"+ter_label)
                        axs[i, j].set_xlabel(soma_label)
                        axs[i, j].set_ylabel(ter_label)
        plt.suptitle('Subtype22_%s_all'%region[0], fontsize=16, fontweight='bold')
        plt.subplots_adjust(wspace=0.25, hspace=0.35) 
        plt.savefig("Subtype22_%s_all.jpg"%region[0],dpi=600)
        plt.savefig("Subtype22_%s_all.pdf"%region[0],dpi=600)
        dfplot.to_csv("subtype22_fig3_plot1_%s.csv"%region[0])
        dfplot2.to_csv("subtype22_fig3_plot2_%s.csv"%region[0])
        plt.close()
                